In [ ]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import datetime
import os
from data_upload_utils import upload_to_github, update_airtable, create_airtable_record, delete_file_from_github

# Initialize CoinGecko
cg = CoinGeckoAPI()

# Define coins
coins = {
    'BTC': 'bitcoin',
    'ETH': 'ethereum',
    'ADA': 'cardano',
    'SOL': 'solana',
    'DOT': 'polkadot',
    'AVAX': 'avalanche-2'
}

# Fetch 365 days of OHLC data and compute metrics
data = []
for symbol, coin_id in coins.items():
    market_data = cg.get_coin_market_chart_by_id(id=coin_id, vs_currency='usd', days=365)
    ohlc_data = market_data['prices']
    
    for i in range(1, len(ohlc_data)):
        # Extract previous day's OHLC data and current day OHLC data
        prev_day = ohlc_data[i-1]
        current_day = ohlc_data[i]
        
        prev_timestamp, prev_price = prev_day
        current_timestamp, current_price = current_day
        
        # Calculate High, Low, Volatility, Trading Range
        high = max(prev_price, current_price)  # Max of the two days
        low = min(prev_price, current_price)   # Min of the two days
        
        volatility = ((high - low) / low) * 100 if low > 0 else 0
        trading_range = high - low
        
        # Store data
        data.append({
            'symbol': symbol,
            'timestamp': datetime.utcfromtimestamp(current_timestamp / 1000),  # Convert timestamp
            'high_24h_usd': high,
            'low_24h_usd': low,
            'volatility_24h_%': round(volatility, 2),
            'trading_range_24h_usd': round(trading_range, 2)
        })

# Create DataFrame
df = pd.DataFrame(data)

# Excel export
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"historical_volatility_trading_range_365_days_{timestamp}.xlsx"
df.to_excel(filename, index=False)

# GitHub + Airtable Upload
AIRTABLE_API_KEY = os.getenv("AIRTABLE_API_KEY")
BASE_ID = "appnssPRD9yeYJJe5"
TABLE_NAME = "Database"
airtable_url = f"https://api.airtable.com/v0/{BASE_ID}/{TABLE_NAME}"
airtable_headers = {
    "Authorization": f"Bearer {AIRTABLE_API_KEY}",
    "Content-Type": "application/json"
}

response = requests.get(airtable_url, headers=airtable_headers)
response.raise_for_status()
data_airtable = response.json()

# Check for existing file
existing_records = [
    rec for rec in data_airtable['records']
    if rec['fields'].get('Name') == "365-Day Volatility and Range"
]

if existing_records and 'Database Attachment' in existing_records[0]['fields']:
    record_id = existing_records[0]['id']
    existing_data = existing_records[0]['fields'].get('Data', [])
else:
    record_id = None
    existing_data = []

# Convert datetime objects to strings (ISO 8601 format) for both new and existing data
for entry in data:
    entry['timestamp'] = entry['timestamp'].isoformat()  # Convert datetime to ISO string

# If appending new data to existing data
new_data_to_append = data  # In this case, it's the same as the current data, but you could compare with the previous set if needed
existing_data.extend(new_data_to_append)

# Prepare Airtable update
if record_id:
    patch_url = f"{airtable_url}/{record_id}"
    patch_payload = {
        "fields": {
            "Name": "365-Day Volatility and Range",
            "Data": existing_data  # Append new data to existing data
        }
    }
    airtable_resp = requests.patch(patch_url, headers=airtable_headers, json=patch_payload)
else:
    post_payload = {
        "records": [{
            "fields": {
                "Name": "365-Day Volatility and Range",
                "Data": existing_data  # Append new data to existing data
            }
        }]
    }
    airtable_resp = requests.post(airtable_url, headers=airtable_headers, json=post_payload)

if airtable_resp.status_code == 200:
    print("✅ Successfully updated Airtable record with new data.")
else:
    raise Exception(f"❌ Airtable update failed: {airtable_resp.status_code} - {airtable_resp.text}")

# Upload to GitHub
GITHUB_REPO = "SagarFieldElevate/DatabaseManagement"
BRANCH = "main"
UPLOAD_PATH = "uploads"
GITHUB_TOKEN = os.getenv("GH_TOKEN")

# Use common function to upload file to GitHub
github_response = upload_to_github(filename, GITHUB_REPO, BRANCH, UPLOAD_PATH, GITHUB_TOKEN)
raw_url = github_response['content']['download_url']  # Get the raw URL from GitHub response
file_sha = github_response['content']['sha']  # Get the SHA for file deletion later

# Update Airtable with GitHub URL
if record_id:
    patch_url = f"{airtable_url}/{record_id}"
    patch_payload = {
        "fields": {
            "Database Attachment": [{
                "url": raw_url,
                "filename": filename
            }]
        }
    }
    airtable_resp = requests.patch(patch_url, headers=airtable_headers, json=patch_payload)
else:
    post_payload = {
        "records": [{
            "fields": {
                "Name": "365-Day Volatility and Range",
                "Database Attachment": [{
                    "url": raw_url,
                    "filename": filename
                }]
            }
        }]
    }
    airtable_resp = requests.post(airtable_url, headers=airtable_headers, json=post_payload)

if airtable_resp.status_code != 200:
    raise Exception(f"❌ Airtable upload failed: {airtable_resp.status_code} - {airtable_resp.text}")

# Delete the file from GitHub after upload
delete_file_from_github(filename, GITHUB_REPO, BRANCH, UPLOAD_PATH, GITHUB_TOKEN, file_sha)

# Delete local file
os.remove(filename)
print("🧽 Local file and GitHub file deleted.")
